# Innodb存储引擎

面向oltp(online transaction processing)、行锁、支持外键、非锁定读、默认采用repeaable级别（可重复读）通过next-keylocking策略避免幻读、插入缓冲、二次写、自适应哈希索引、预读


## Innodb特性

主体系结构：默认7个后台线程，4个io thread(insert buffer、log、read、write),1个master thread(优先级最高),1个锁(lock)监控线程，1个错误监控线程。可以通过show engine innodb status来查看。新版本已对默认的read thread和write thread分别增大到4个，可通过show variables like 'innodb_io_thread%'查看。

存储引擎组成：缓冲池(buffer pool)、重做日志缓冲池(redo log buffer)以及额外的内存池(additional memory pool).具体配置可由show variables like 'innodb_buffer_pool_size'、show variables like'innodb_log_buffer_size'、show variables like 'innodb_additional_mem_pool_size'来查看。

缓冲池：占最大块内存，用来存放各种数据的缓存包括有索引页、数据页、undo页、插入缓冲、自适应哈希索引、innodb存储的锁信息、数据字典信息等。工作方式总是将数据库文件按页(每页16k)读取到缓冲池，然后按最近最少使用(lru)的算法来保留在缓冲池中的缓存数据。如果数据库文件需要修改，总是首先修改在缓存池中的页(发生修改后即为脏页)，然后再按照一定的频率将缓冲池的脏页刷新到文件。通过命令show engine innodb status;来查看。 日志缓冲：将重做日志信息先放入这个缓冲区，然后按一定频率将其刷新到重做日志文件。

master thread: 
loop主循环每秒一次的操作： 
日志缓冲刷新到磁盘，即使这个事务还没有提交。(总是执行，所以再大的事务commit 的时间也是很快的) 
合并插入缓冲(innodb当前一秒发生的io次数小于5次则执行) 
至多刷新100个innodb的缓冲池中的脏页到磁盘(超过配置的脏页所占缓冲池比例则执行，在配置文件中由innodb_max_dirty_pages_pac决定，默认是90，新版本是75，google建议是80) 
如果当前没用用户活动，切换到backgroud loop

5．优化的 SQL查询算法，有效地提高查询速度。

6．既能够作为一个单独的应用程序应用在客户端服务器网络环境中，也能够作为一个库而嵌入到其他的软件中。

7．提供多语言支持，常见的编码如中文的 GB 2312、BIG5，日文的 Shift_JIS等都可以用作数据表名和数据列名。

8．提供 TCP/IP、ODBC 和 JDBC等多种数据库连接途径。

9．提供用于管理、检查、优化数据库操作的管理工具。

10．支持大型的数据库。可以处理拥有上千万条记录的大型数据库。

11．支持多种存储引擎。

12．MySQL 是开源的，所以你不需要支付额外的费用。

13．MySQL 使用标准的 SQL数据语言形式。

14．MySQL 对 PHP 有很好的支持，PHP是目前最流行的 Web 开发语言。

15．MySQL是可以定制的，采用了 GPL协议，你可以修改源码来开发自己的 MySQL 系统。

##MySQL各个存储引擎

innodb存储引擎：面向oltp(online transaction processing)、行锁、支持外键、非锁定读、默认采用repeaable级别（可重复读）通过next-keylocking策略避免幻读、插入缓冲、二次写、自适应哈希索引、预读

myisam存储引擎：不支持事务、表锁、全文索引、适合olap（在线分析处理），其中myd:放数据文件，myi:放索引文件

ndb存储引擎：集群存储引擎，share nothing，可提高可用性

memory存储引擎：数据存放在内存中，表锁，并发性能差，默认使用哈希索引

archive存储引擎：只支持insert和select zlib算法压缩1：10，适合存储归档数据如日志等、行锁

maria存储引擎：目的取代myisam、缓存数据和索引、行锁、mvcc

<img align="left" style="padding-right:10px;" src="figures/mysql-storage.png">


## MySQL 文件

参数文件：告诉Mysql实例启动时在哪里可以找到数据库文件，并且指定某些初始化参数，这些参数定义了某种内存结构的大小等设置。用文件存储，可编辑，若启动时加载不到则不能成功启动（与其他数据库不同）。参数有动态和静态之分，静态相当于只读，动态是可以set的。如我们通过show variable like '***'查出来的key、value值，是可以通过set key=value直接修改的。同是，修改时还有作用域之分，即这个seesion个有效和全局有效，在对应的key前加上session或global即可，如select @@seesion.read_buffer_size、set @@global.read_buffer_size。

日志文件：用来记录Mysql实例对某种条件做出响应时写入的文件。如错误日志文件、二进制日志文件、慢查询日志文件、查询日志文件等。

错误日志：通过show variables like 'log_error'来查看错误日志存放地址。

慢查询日志：通过show variables like '%long%' 查看慢查询日志记录的阈值，新版本设成了0.05；通过show variables like 'log_slow_queries'查看是否开启了，默认为关闭的；通过show variabes like 'log_queries_not_using_indexes'查看是将没有使用索引的查询记录到慢日志中。mysql中可以直接通过mysqldumpslow命令来查看慢日志。

二进制文件：不记录查询，只记录对数据库所有的修改操作。目的是为了恢复(point-in-time修复)和复制。通过show variables like 'datadir'查看存放路径。二进制日志支持STATEMENT、ROW、MIX三种格式，通过binlog_format参数设定，通常设置为ROW，可以为数据库的恢复和复制带来更好的可靠性，但会带来二进制文件大小的增加，复制时会增加网络开销。mysql中通过mysqlbinlog查看二进制日志文件内容。

socket文件：当用Unix域套接字方式进行连接时需要的文件。

pid文件：Mysql实例的进程ID文件。 

Mysql表结构文件：用来存放Mysql表结构定义文件。因为Mysql插件式存储引擎的体系结构，每个表都有一个对应的文件，以frm后缀结尾。

存储引擎文件：存储自己的文件来保存各种数据，真正存储了数据和索引等数据。下面主要介绍InnoDB的存储引擎下的表空间文件和重做日志文件。

表空间文件：InnoDB默认的表空间文件为ibdata1,可通过show variables like 'innodb_file_per_table'查看每个表是否产生单独的.idb表空间文件。但是，单独的表空间文件仅存储该表的数据、索引和插入缓冲等信息，其余信息还是存放在默认的表空间中。

重做日志文件：实例和介质失败，重做日志文件就能派上用场，如数据库掉电，InnoDB存储引擎会使用重做日志恢复到掉电前的时刻，以此来保证数据的完整性。参数innodb_log_file_size指定了重做日志文件的大小；innodb_log_file_in_group指定了日志文件组中重做日志文件的数量，默认为2，innodb_mirrored_log_groups指定了日志镜像文件组的数量，默认为1，代表只有一个日志文件组，没有镜像；innodb_log_group_home_dir指定了日志文件组所在路径，默认在数据库路径下。
